# First model

## Setup

#### Imports

In [1]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import pathlib
import logging
import sys

from amftrack.util.sys import storage_path
from amftrack.util.file import chose_file

2022-05-16 13:36:12.007220: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-16 13:36:12.007235: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/ipausers/kahane/Wks/AMFtrack/amftrack/util/dbx.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


**Notebook logger**

In [2]:
logger = logging.getLogger("notebook")
logger.setLevel(logging.DEBUG)

**Paths**

In [114]:
tf.config.functions_run_eagerly()

False

In [115]:
tf.config.run_functions_eagerly(True)

In [4]:
dataset_path = os.path.join(storage_path, "width3", "dataset_2")

In [5]:
dataset_path

'/media/kahane/AMFtopology02/storage/width3/dataset_2'

In [6]:
section_path = os.path.join(dataset_path, "Img")

In [7]:
edge_data_path = os.path.join(dataset_path, "Data")

#### Utils

In [8]:
def get_nodes(ch):
    nodes = ch.split("-")
    print(nodes)
    return nodes[0], nodes[1]

**Variables**

In [9]:
BATCHSIZE = 32
BUFFERSIZE = 12
INPUTSIZE = 120
SHUFFLE_BUFFER = 10

## I/ Data pipeline

Some remarks:

- We use different edges for training validation and test

In [116]:
def image_path_to_df_path(path: str):
    image_name = os.path.basename(str(path))
    image_name_without_ext = os.path.splitext(image_name)[0]
    df_name = image_name_without_ext + ".csv"
    edge_data_full_path = os.path.join(edge_data_path, df_name)
    return edge_data_full_path

In [117]:
txt = '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1122-1227.png'
image_path_to_df_path(txt)

'/media/kahane/AMFtopology02/storage/width3/dataset_2/Data/1122-1227.csv'

In [118]:
# TODO(FK): make tf function

#@tf.function
def load_image(filename):
    "From one file name (corresponding to an edge). Loads the data associated with this one file/edge."
    logger.info(f"Type of file: {type(filename)}")
    logger.info(f"File name: {filename}")
    print(filename)
    # 1/ Slices from the edge
    raw = tf.io.read_file(filename) # open the file
    image = tf.image.decode_png(raw, channels=1, dtype = tf.uint8)
    logger.debug(f"Initial shape: {image.shape}")
    # image = image.set_shape([None, 120, 1]) 
    image = tf.squeeze(image) # removing the last axis
    # TODO (FK): chose here only part of the array
    logger.debug(f"Final shape: {image.shape}")
    # TODO (FK): why don't I get the shape
    slice_dataset = tf.data.Dataset.from_tensor_slices(image)

    # 2/ Information on the edge
    # TODO (FK): verify order for the edge dataframe
    #edge_name = os.path.splitext(os.path.basename(str(filename)))[0] + ".csv" # PB: not a tensor
    #edge_data_full_path = os.path.join(edge_data_path, edge_name)
    #edge_data_full_path = image_path_to_df_path(filename)
    edge_data_full_path = tf.map_fn(filename, image_path_to_df_path)
    logger.debug(edge_data_full_path)
    edge_df = pd.read_csv(edge_data_full_path)
    logger.debug(edge_df.columns)
    feature_dataset = edge_df[['x1_image', 'y2_image']]
    
    # 3/ Labels
    label_dataset = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor(edge_df[['width']]))

    return tf.data.Dataset.zip((slice_dataset, label_dataset))
    #return slice_dataset

In [119]:
b = load_image(chose_file(section_path))

2022-05-16 14:54:43,635-[INFO]- notebook:6 -> Type of file: <class 'str'>
2022-05-16 14:54:43,635-[INFO]- notebook:7 -> File name: /media/kahane/AMFtopology02/storage/width3/dataset_2/Img/826-644.png
/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/826-644.png
2022-05-16 14:54:44,170-[DEBUG]- notebook:12 -> Initial shape: (136, 120, 1)
2022-05-16 14:54:44,172-[DEBUG]- notebook:16 -> Final shape: (136, 120)


AttributeError: 'str' object has no attribute '__name__'

In [ ]:
for elem in b:
    print(elem)

In [ ]:
c = b.shuffle(100)

In [120]:
path_dataset = tf.data.Dataset.list_files(filepaths)

In [121]:
general_dataset = path_dataset.interleave(load_image, cycle_length=3)

2022-05-16 14:54:46,053-[INFO]- notebook:446 -> Type of file: <class 'tensorflow.python.framework.ops.Tensor'>
2022-05-16 14:54:46,071-[INFO]- notebook:446 -> File name: Tensor("args_0:0", shape=(), dtype=string)
Tensor("args_0:0", shape=(), dtype=string)
2022-05-16 14:54:46,090-[DEBUG]- notebook:446 -> Initial shape: (None, None, 1)
2022-05-16 14:54:46,105-[DEBUG]- notebook:446 -> Final shape: <unknown>


/home/ipausers/kahane/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


AttributeError: in user code:

    File "/tmp/ipykernel_5119/3011409102.py", line 25, in load_image  *
        edge_data_full_path = tf.map_fn(filename, image_path_to_df_path)

    AttributeError: 'Tensor' object has no attribute '__name__'


In [ ]:
general_dataset = general_dataset.shuffle(shuffle_buffer_size).repeat(repeat)

In [57]:
def reader_dataset(filepaths, repeat=1, n_readers=5, shuffle_buffer_size = 1000, batch_size=32):
    """
    Take as input a list of the paths to the data files.
    And return a tf.Dataset object iterating through the batches.
    """
    path_dataset = tf.data.Dataset.list_files(filepaths) # yield file names randomly
    # TODO: make buffer size to size of the dataset
    # TODO: get a dataset object from one file
    general_dataset = path_dataset.interleave(lambda filepath: load_image(filepath), cycle_length=n_readers)
    general_dataset = general_dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return general_dataset.batch(batch_size).prefetch(1)

In [55]:
filepaths = [os.path.join(section_path, file) for file in os.listdir(section_path)]

In [83]:
filepaths

['/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/2413-2729.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1122-1227.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1211-1227.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1227-1298.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1255-1458.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1458-1316.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1458-1686.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1538-1502.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1686-1723.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1723-1787.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1787-1889.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/1841-1391.png',
 '/media/kahane/AMFtopology02/storage/width3/dataset_2/Img/185-369.png',
 '/media/kahane/AMFtopology

In [56]:
reader_dataset(filepaths)

UnboundLocalError: local variable 'general_dataset' referenced before assignment

In [18]:
a = reader_dataset(filepaths)
for e in a:
    print(e.shape)

(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)
(32, 120)


In [13]:
data = pd.read_csv(os.path.join(dataset_path, "data.csv")) #.set_index('edge')
#del data['Unnamed: 0']

In [17]:
data[data["edge"] == f"(Node({482}),Node({478}))"]

,Unnamed: 0,edge,width


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  43 non-null     int64  
 1   edge        43 non-null     object 
 2   width       43 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ KB


### 2/ Coordinates

## II/ First model

# Brouillon

In [12]:
image_dir = 'hypothesis/temp'
image_root = pathlib.Path(image_dir)
list_ds = tf.data.Dataset.list_files(str(image_root/'*.png'))
for f in list_ds:
  image = tf.io.read_file(f)
  image = tf.io.decode_png(image)

NameError: name 'pathlib' is not defined

In [ ]:
def process_path(file_path):
  label = tf.strings.split(file_path, os.sep)[-2]
  return tf.io.read_file(file_path), label

labeled_ds = list_ds.map(process_path)

for image_raw, label_text in labeled_ds.take(1):
  print(repr(image_raw.numpy()[:100]))
  print()
  print(label_text.numpy())


In [ ]:
@tf.function
def load_image(filename):
    raw = tf.io.read_file(filename)
    image = tf.image.decode_png(raw, channels=1)
    # the `print` executes during tracing.
    print("Initial shape: ", image.shape)
    image.set_shape([28, 28, 3])
    print("Final shape: ", image.shape)
    return image


In [ ]:
def process_path(file_path):
    label = tf.strings.split(file_path, os.sep)[-2]

In [ ]:
def to_dataset(dataframe, shuffle=True, batch_size=32):
    labels = dataframe.pop('label')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [ ]:
import tensorflow as tf
filenames = ["./img1.jpg", "./img2.jpg"]
labels = ["A", "B"]

def load_image(filePath, label):
    print('Loading File: {}' + filePath)
    raw_bytes = tf.io.read_file(filePath)
    image = tf.io.decode_image(raw_bytes, expand_animations = False)
    return tf.data.Dataset.from_tensors((image, label))

dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.interleave(lambda x, y: load_image(x, y), cycle_length=4)

for i in dataset.as_numpy_iterator():
    image = i[0]
    label = i[1]
    print(image.shape)
    print(label.decode())

In [ ]:
import csv

with open('coors.csv', mode='r') as infile:
    reader = csv.reader(infile)